чтение и анализ логов бэнчмарка

# Импорты

In [1]:
import re
from pathlib import Path
from pprint import pprint
from collections import defaultdict

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
import seaborn as sns
from plotly import colors
from sklearn.decomposition import PCA
from sklearn.preprocessing import LabelEncoder, MinMaxScaler

from src.utils.read_logs import LogReader

In [2]:
plt.ioff()
%matplotlib notebook

# Чтение и преобразование логов

In [3]:
log_file_path = "logs/2025-06-25 method_logs.json"
logs = LogReader.load_logs_from_file(log_file_path)

In [4]:
len(logs)

10158

In [5]:
logs[0]

{'method_name': 'TensorLy_Tucker_image-0_pytorch_truncated_svd_svd',
 'method_args': {'rank': [252, 255, 3],
  'n_iter_max': 100,
  'init': 'svd',
  'svd': 'truncated_svd',
  'random_state': 42},
 'qualitative_metrics': {'Language': 'Python',
  'Library': 'TensorLy',
  'TensorLy backend': 'pytorch',
  'Tensor type': 'Dense',
  'Data type': 'image-0',
  'Platform': 'CPU, GPU',
  'Decomposition method': 'Tucker'},
 'quantitative_metrics': {'gpu_allocated_memory_used_mb': [23.048828125,
   19.67822265625,
   19.55224609375,
   19.67822265625,
   19.703125],
  'ram_mem_used_mb': [6175.27734375, 6184.375, 6184.375, 6184.375, 6184.375],
  'duration': [4.248620055000174,
   1.7748750900000232,
   1.7735690550000527,
   1.7700060310000936,
   1.9719900439999947],
  'frobenius_error': [0.6721850484609604,
   0.6721850484609604,
   0.6721850484609604,
   0.6721850484609604,
   0.6721850484609604],
  'compression_ratio': [50.166930737890446,
   50.166930737890446,
   50.166930737890446,
   50.166

In [6]:
clean_rows, anom_rows, error_rows = [], [], []
neg_stats = defaultdict(lambda: {"idx": [], "vals": []})  # статистика по аномалиям

metric_keys = [  # порядок важен для синхронного среза
    "gpu_allocated_memory_used_mb",
    "ram_mem_used_mb",
    "duration",
    "frobenius_error",
    "compression_ratio",
]

for log in logs:
    # --- базовая (qualitative) часть ---------------------------------
    base = {
        "method_name": log["method_name"],
        "decomposition_method": log["qualitative_metrics"]["Decomposition method"],
        "data_type": log["qualitative_metrics"]["Data type"],
        "language": log["qualitative_metrics"]["Language"],
        "library": log["qualitative_metrics"]["Library"],
        "tensor_type": log["qualitative_metrics"]["Tensor type"],
        "platform": log["qualitative_metrics"]["Platform"],
        "backend": log["qualitative_metrics"][
            next(k for k in log["qualitative_metrics"] if "backend" in k)
        ],
    }

    # ----- случаи с ошибкой выполнения --------------------------------
    if log["error_message"]:
        error_rows.append(
            {
                **base,
                "error_message": log["error_message"],
                "is_error": True,
            }
        )
        continue
    else:
        error_rows.append(
            {
                **base,
                "error_message": log["error_message"],
                "is_error": False,
            }
        )

    q = log["quantitative_metrics"]
    n_runs = max(len(q[k]) for k in metric_keys)  # обычно 5

    # 1. определяем, какие индексы аномальны
    bad_idx = {i for k in metric_keys
               for i, v in enumerate(q[k])
               if isinstance(v, (int, float)) and v < 0}

    # 2. «чистые» и «плохие» индексы
    good_idx = sorted(set(range(n_runs)) - bad_idx)
    bad_idx = sorted(bad_idx)


    # функция-хелпер: формирует агрегаты для выбранного подмножества индексов
    def make_row(idx_subset, tag):
        if not idx_subset:
            return None  # нечего добавлять
        row = {**base,
               "run_type": tag,  # clean | anomaly
               "original_run_indices": idx_subset}  # список позиций
        for k in metric_keys:
            vals = [q[k][i] for i in idx_subset]
            row[f"{k}_min"] = np.min(vals)
            row[f"{k}_max"] = np.max(vals)
            row[f"{k}_mean"] = np.mean(vals)
            row[f"{k}_median"] = np.median(vals)
        return row


    # 3. сохраняем строки
    r_clean = make_row(good_idx, "clean")
    r_anom = make_row(bad_idx, "anomaly")

    if r_clean: clean_rows.append(r_clean)
    if r_anom:
        anom_rows.append(r_anom)
        # для общей статистики: сколько раз каждая позиция «проваливается»
        for k in metric_keys:
            neg_stats[k]['idx'].extend([i for i in bad_idx
                                        if isinstance(q[k][i], (int, float)) and q[k][i] < 0])
            neg_stats[k]['vals'].extend([q[k][i] for i in bad_idx
                                         if isinstance(q[k][i], (int, float)) and q[k][i] < 0])

# --- превращаем во фреймы --------------------------------------------
df_clean_logs = pd.DataFrame(clean_rows)  # положительные значения прогонов
df_anomaly_logs = pd.DataFrame(anom_rows)  # отрицательные значения прогонов
df_logs_errors = pd.DataFrame(error_rows)  # все логи без количественных метрик

In [7]:
print("metric                         | count_neg |    min   |   mean  |   max   | median ")
print("-" * 100)

for metric, d in neg_stats.items():
    if not d["vals"]:
        continue

    print(
        f"{metric:<30} | "
        f"{len(d['vals']):>9} | "
        f"{np.min(d['vals']):^8.4f} | "
        f"{np.mean(d['vals']):>6.4f} | "
        f"{np.max(d['vals']):>6.4f} | "
        f"{np.median(d['vals']):>6.4f}"
    )


metric                         | count_neg |    min   |   mean  |   max   | median 
----------------------------------------------------------------------------------------------------


In [8]:
# log_data = []
# log_errors = []
# 
# negative_stats = defaultdict(lambda: {
#     "count": 0,
#     "min": float("inf"),
#     "max": float("-inf"),
#     "sum": 0.0
# })
# 
# for log in logs:
#     if log["error_message"] == "":
#         try:
#             data_entry = {
#                 # Method name
#                 "method_name": log["method_name"],
#                 # Some Qualitative Data
#                 "decomposition_method": log["qualitative_metrics"]["Decomposition method"],
#                 "data_type": log["qualitative_metrics"]["Data type"],
#                 "language": log["qualitative_metrics"]["Language"],
#                 "library": log["qualitative_metrics"]["Library"],
#                 "tensor_type": log["qualitative_metrics"]["Tensor type"],
#                 "platform": log["qualitative_metrics"]["Platform"],
#                 "backend": log["qualitative_metrics"].get(
#                     next(key for key in log["qualitative_metrics"] if "backend" in key)
#                 ),
#                 # GPU Allocated Memory
#                 "gpu_allocated_memory_used_min": np.min(
#                     [e for e in log["quantitative_metrics"]["gpu_allocated_memory_used_mb"] if e >= 0.0]
#                 ),
#                 "gpu_allocated_memory_used_max": np.max(
#                     [e for e in log["quantitative_metrics"]["gpu_allocated_memory_used_mb"] if e >= 0.0]
#                 ),
#                 "gpu_allocated_memory_used_mean": np.mean(
#                     [e for e in log["quantitative_metrics"]["gpu_allocated_memory_used_mb"] if e >= 0.0]
#                 ),
#                 "gpu_allocated_memory_used": np.max(
#                     [e for e in log["quantitative_metrics"]["gpu_allocated_memory_used_mb"] if e >= 0.0]
#                 ),
#                 # GPU Cached Memory
#                 "gpu_cached_memory_used_min": np.min(
#                     [e for e in log["quantitative_metrics"]["gpu_cached_memory_used_mb"] if e >= 0.0]
#                 ),
#                 "gpu_cached_memory_used_max": np.max(
#                     [e for e in log["quantitative_metrics"]["gpu_cached_memory_used_mb"] if e >= 0.0]
#                 ),
#                 "gpu_cached_memory_used_mean": np.mean(
#                     [e for e in log["quantitative_metrics"]["gpu_cached_memory_used_mb"] if e >= 0.0]
#                 ),
#                 "gpu_cached_memory_used": np.max(
#                     [e for e in log["quantitative_metrics"]["gpu_cached_memory_used_mb"] if e >= 0.0]
#                 ),
#                 # RAM Memory Usage
#                 "ram_mem_used_min": np.min([e for e in log["quantitative_metrics"]["ram_mem_used_mb"] if e >= 0.0]),
#                 "ram_mem_used_max": np.max([e for e in log["quantitative_metrics"]["ram_mem_used_mb"] if e >= 0.0]),
#                 "ram_mem_used_mean": np.mean([e for e in log["quantitative_metrics"]["ram_mem_used_mb"] if e >= 0.0]),
#                 "ram_mem_used": np.max([e for e in log["quantitative_metrics"]["ram_mem_used_mb"] if e >= 0.0]),
#                 # Duration
#                 "duration_min": np.min([e for e in log["quantitative_metrics"]["duration"] if e >= 0.0]),
#                 "duration_max": np.max([e for e in log["quantitative_metrics"]["duration"] if e >= 0.0]),
#                 "duration_mean": np.mean([e for e in log["quantitative_metrics"]["duration"] if e >= 0.0]),
#                 "duration": np.max([e for e in log["quantitative_metrics"]["duration"] if e >= 0.0]),
#                 # Frobenius Error
#                 "frobenius_error_min": np.min([e for e in log["quantitative_metrics"]["frobenius_error"] if e >= 0.0]),
#                 "frobenius_error_max": np.max([e for e in log["quantitative_metrics"]["frobenius_error"] if e >= 0.0]),
#                 "frobenius_error_mean": np.mean(
#                     [e for e in log["quantitative_metrics"]["frobenius_error"] if e >= 0.0]
#                 ),
#                 "frobenius_error": np.max([e for e in log["quantitative_metrics"]["frobenius_error"] if e >= 0.0]),
#                 # Compression Ratio
#                 "compression_ratio_min": np.min(
#                     [e for e in log["quantitative_metrics"]["compression_ratio"] if e >= 0.0]
#                 ),
#                 "compression_ratio_max": np.max(
#                     [e for e in log["quantitative_metrics"]["compression_ratio"] if e >= 0.0]
#                 ),
#                 "compression_ratio_mean": np.mean(
#                     [e for e in log["quantitative_metrics"]["compression_ratio"] if e >= 0.0]
#                 ),
#                 "compression_ratio": np.max([e for e in log["quantitative_metrics"]["compression_ratio"] if e >= 0.0]),
#             }
#             log_data.append(data_entry)
# 
#             for key, values in log["quantitative_metrics"].items():
#                 if not isinstance(values, list):
#                     continue
# 
#                 neg_values = [v for v in values if isinstance(v, (int, float)) and v < 0]
#                 if neg_values:
#                     print(f"[{log['method_name']}] Key '{key}' has negative values: {neg_values}")
# 
#                     negative_stats[key]["count"] += len(neg_values)
#                     negative_stats[key]["min"] = min(negative_stats[key]["min"], min(neg_values))
#                     negative_stats[key]["max"] = max(negative_stats[key]["max"], max(neg_values))
#                     negative_stats[key]["sum"] += sum(neg_values)
# 
# 
#         except Exception as e:
#             print(f"Exception with {log['method_name']}. Error: {e!s}")
#     else:
#         data_entry = {
#             # method name
#             "method_name": log["method_name"],
#             # qualitative metrics
#             "decomposition_method": log["qualitative_metrics"]["Decomposition method"],
#             "data_type": log["qualitative_metrics"]["Data type"],
#             "language": log["qualitative_metrics"]["Language"],
#             "library": log["qualitative_metrics"]["Library"],
#             "tensor_type": log["qualitative_metrics"]["Tensor type"],
#             "platform": log["qualitative_metrics"]["Platform"],
#             "backend": log["qualitative_metrics"].get(
#                 next(key for key in log["qualitative_metrics"] if "backend" in key)
#             ),
#             # Error data
#             "error_message": log["error_message"],
#         }
#         log_errors.append(data_entry)
# 
# df_logs = pd.DataFrame(log_data)
# df_logs_errors = pd.DataFrame(log_errors)

In [9]:
# print("\n=== Summary of Negative Values in Quantitative Metrics ===")
# for key, stats in negative_stats.items():
#     count = stats["count"]
#     if count > 0:
#         avg = stats["sum"] / count
#         print(f"Key: {key}")
#         print(f"  → Count   : {count}")
#         print(f"  → Min     : {stats['min']}")
#         print(f"  → Max     : {stats['max']}")
#         print(f"  → Mean    : {avg:.4f}\n")

In [10]:
df_clean_logs.shape

(4772, 30)

In [11]:
# for col in df_clean_logs.columns:
#     try:
#         unique_vals = df_clean_logs[col].unique()
#         print(f"Column: {col}")
#         print(f"Unique values ({len(unique_vals)}): {unique_vals}")
#         print("-" * 60)
#     except TypeError as e:
#         print(f"Skipped column '{col}' due to unhashable data type: {e}")

In [12]:
df_clean_logs

,method_name,decomposition_method,data_type,language,library,tensor_type,platform,backend,run_type,original_run_indices,...,duration_mean,duration_median,frobenius_error_min,frobenius_error_max,frobenius_error_mean,frobenius_error_median,compression_ratio_min,compression_ratio_max,compression_ratio_mean,compression_ratio_median
0,TensorLy_Tucker_image-0_pytorch_truncated_svd_svd,Tucker,image-0,Python,TensorLy,Dense,"CPU, GPU",pytorch,clean,"[0, 1, 2, 3, 4]",...,2.307812,1.774875,0.672185,0.672185,0.672185,0.672185,50.166931,50.166931,50.166931,50.166931
1,TensorLy_Tucker_image-0_pytorch_truncated_svd_...,Tucker,image-0,Python,TensorLy,Dense,"CPU, GPU",pytorch,clean,"[0, 1, 2, 3, 4]",...,2.658001,2.638634,0.672281,0.672281,0.672281,0.672281,50.166931,50.166931,50.166931,50.166931
2,TensorLy_Tucker_image-0_pytorch_symeig_svd_random,Tucker,image-0,Python,TensorLy,Dense,"CPU, GPU",pytorch,clean,"[0, 1, 2, 3, 4]",...,2.753213,2.702102,0.672281,0.672281,0.672281,0.672281,50.166931,50.166931,50.166931,50.166931
3,TensorLy_Tucker_image-0_pytorch_randomized_svd...,Tucker,image-0,Python,TensorLy,Dense,"CPU, GPU",pytorch,clean,"[0, 1, 2, 3, 4]",...,1.373213,1.388822,0.672679,0.672679,0.672679,0.672679,50.166931,50.166931,50.166931,50.166931
4,TensorLy_Tucker_image-0_pytorch_randomized_svd...,Tucker,image-0,Python,TensorLy,Dense,"CPU, GPU",pytorch,clean,"[0, 1, 2, 3, 4]",...,2.717210,2.757429,0.672281,0.672281,0.672281,0.672281,50.166931,50.166931,50.166931,50.166931
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4767,T3F_TensorTrain_image-1,TensorTrain,image-1,Python,T3F,Dense,"CPU, GPU",tensorflow,clean,"[0, 1, 2, 3, 4]",...,0.408760,0.386177,3.427543,3.427543,3.427543,3.427543,50.229930,50.229930,50.229930,50.229930
4768,T3F_TensorTrain_image-2,TensorTrain,image-2,Python,T3F,Dense,"CPU, GPU",tensorflow,clean,"[0, 1, 2, 3, 4]",...,0.479950,0.481683,0.000883,0.000883,0.000883,0.000883,50.006579,50.006579,50.006579,50.006579
4769,T3F_TensorTrain_video-0,TensorTrain,video-0,Python,T3F,Dense,"CPU, GPU",tensorflow,clean,"[0, 1, 2, 3, 4]",...,0.603289,0.569417,0.338334,0.338334,0.338334,0.338334,50.340601,50.340601,50.340601,50.340601
4770,T3F_TensorTrain_video-1,TensorTrain,video-1,Python,T3F,Dense,"CPU, GPU",tensorflow,clean,"[0, 1, 2, 3, 4]",...,0.462598,0.436068,0.477660,0.477660,0.477660,0.477660,50.094968,50.094968,50.094968,50.094968


In [13]:
df_anomaly_logs.shape

(0, 0)

In [14]:
# for col in df_anomaly_logs.columns:
#     try:
#         unique_vals = df_anomaly_logs[col].unique()
#         print(f"Column: {col}")
#         print(f"Unique values ({len(unique_vals)}): {unique_vals}")
#         print("-" * 60)
#     except TypeError as e:
#         print(f"Skipped column '{col}' due to unhashable data type: {e}")

In [15]:
df_anomaly_logs

""


In [16]:
df_logs_errors.shape

(10158, 10)

In [17]:
# for col in df_logs_errors.columns:
#     try:
#         unique_vals = df_logs_errors[col].unique()
#         print(f"Column: {col}")
#         print(f"Unique values ({len(unique_vals)}): {unique_vals}")
#         print("-" * 60)
#     except TypeError as e:
#         print(f"Skipped column '{col}' due to unhashable data type: {e}")

In [18]:
df_logs_errors

,method_name,decomposition_method,data_type,language,library,tensor_type,platform,backend,error_message,is_error
0,TensorLy_Tucker_image-0_pytorch_truncated_svd_svd,Tucker,image-0,Python,TensorLy,Dense,"CPU, GPU",pytorch,,False
1,TensorLy_Tucker_image-0_pytorch_truncated_svd_...,Tucker,image-0,Python,TensorLy,Dense,"CPU, GPU",pytorch,,False
2,TensorLy_Tucker_image-0_pytorch_symeig_svd_svd,Tucker,image-0,Python,TensorLy,Dense,"CPU, GPU",pytorch,"Пропущена итерация из-за ошибки: pytorch, syme...",True
3,TensorLy_Tucker_image-0_pytorch_symeig_svd_random,Tucker,image-0,Python,TensorLy,Dense,"CPU, GPU",pytorch,,False
4,TensorLy_Tucker_image-0_pytorch_randomized_svd...,Tucker,image-0,Python,TensorLy,Dense,"CPU, GPU",pytorch,,False
...,...,...,...,...,...,...,...,...,...,...
10153,T3F_TensorTrain_image-1,TensorTrain,image-1,Python,T3F,Dense,"CPU, GPU",tensorflow,,False
10154,T3F_TensorTrain_image-2,TensorTrain,image-2,Python,T3F,Dense,"CPU, GPU",tensorflow,,False
10155,T3F_TensorTrain_video-0,TensorTrain,video-0,Python,T3F,Dense,"CPU, GPU",tensorflow,,False
10156,T3F_TensorTrain_video-1,TensorTrain,video-1,Python,T3F,Dense,"CPU, GPU",tensorflow,,False


## Обработка ошибок

In [19]:
def extract_memory_info(error_message):
    pattern = r"(\d+\.?\d*)\s*(\w+iB)"
    match = re.search(pattern, error_message)

    if match:
        value = float(match.group(1))  # Память
        unit = match.group(2)  # Тип памяти (единица измерения)

        # Конвертация в MiB
        conversion_factors = {
            "KiB": 1 / 1024,  # Килобайты -> Мегабайты
            "MiB": 1,  # Мегабайты -> Мегабайты
            "GiB": 1024,  # Гигабайты -> Мегабайты
            "TiB": 1024 ** 2,  # Терабайты -> Мегабайты
            "PiB": 1024 ** 3,  # Петабайты -> Мегабайты
            "EiB": 1024 ** 4,  # Эксабайты -> Мегабайты
            "ZiB": 1024 ** 5,  # Зеттабайты -> Мегабайты
            "YiB": 1024 ** 6,  # Йоттабайты -> Мегабайты
        }

        if unit in conversion_factors:
            value_in_mib = value * conversion_factors[unit]
        else:
            raise ValueError(f"Неизвестная единица измерения памяти: {unit}")

        return value_in_mib, "MiB"

    return None, None

In [20]:
error_messages_count = 0

df_logs_errors_without_singular_matrix = pd.DataFrame()
df_logs_errors_only_singular_matrix = pd.DataFrame()

for index, row in df_logs_errors.iterrows():
    row_copy = row.copy()
    error_message = row_copy["error_message"]

    if "Tried to allocate" in error_message:  # pytorch
        value, unit = extract_memory_info(error_message)
        if not (value and unit):
            print("Не удалось извлечь информацию о памяти из GPU and RAM ошибки.")
    elif "Unable to allocate" in error_message:  # numpy
        value, unit = extract_memory_info(error_message)
        if not (value and unit):
            print("Не удалось извлечь информацию о памяти из RAM ошибки.")
    elif row_copy["error_message"] == "":
        value = 0.0
    else:
        value = None
        error_messages_count += 1
        print(error_message)

    if value is not None:
        row_copy["memory_tried_to_allocated_error_mb"] = value
        df_logs_errors_without_singular_matrix = pd.concat(
            [df_logs_errors_without_singular_matrix, pd.DataFrame([row_copy])], ignore_index=True)
    else:
        df_logs_errors_only_singular_matrix = pd.concat([df_logs_errors_only_singular_matrix, pd.DataFrame([row_copy])],
                                                        ignore_index=True)

print(error_messages_count)

Пропущена итерация из-за ошибки: pytorch, truncated_svd, random, False, False, 1e-08, 0, abs_rec_error. Ошибка: torch.linalg.solve: The solver failed because the input matrix is singular.
Пропущена итерация из-за ошибки: pytorch, symeig_svd, random, False, False, 1e-08, 0, abs_rec_error. Ошибка: torch.linalg.solve: The solver failed because the input matrix is singular.
Пропущена итерация из-за ошибки: pytorch, randomized_svd, random, False, False, 1e-08, 0, abs_rec_error. Ошибка: torch.linalg.solve: The solver failed because the input matrix is singular.
Пропущена итерация из-за ошибки: pytorch, truncated_svd, random, False, False, 1e-05, 0, abs_rec_error. Ошибка: torch.linalg.solve: The solver failed because the input matrix is singular.
Пропущена итерация из-за ошибки: pytorch, symeig_svd, random, False, False, 1e-05, 0, abs_rec_error. Ошибка: torch.linalg.solve: The solver failed because the input matrix is singular.
Пропущена итерация из-за ошибки: pytorch, randomized_svd, random,

In [21]:
print(
    df_logs_errors.shape,
    df_logs_errors.columns,
    df_logs_errors_without_singular_matrix.shape,
    df_logs_errors_without_singular_matrix.columns,
    df_logs_errors_only_singular_matrix.shape,
    df_logs_errors_only_singular_matrix.columns,
    sep="\n"
)

(10158, 10)
Index(['method_name', 'decomposition_method', 'data_type', 'language',
       'library', 'tensor_type', 'platform', 'backend', 'error_message',
       'is_error'],
      dtype='object')
(9828, 11)
Index(['method_name', 'decomposition_method', 'data_type', 'language',
       'library', 'tensor_type', 'platform', 'backend', 'error_message',
       'is_error', 'memory_tried_to_allocated_error_mb'],
      dtype='object')
(330, 10)
Index(['method_name', 'decomposition_method', 'data_type', 'language',
       'library', 'tensor_type', 'platform', 'backend', 'error_message',
       'is_error'],
      dtype='object')


# Функции для анализа логов

In [22]:
# Отрисовка горизонтальных бар-графиков для метрик
def plot_barh(ax, x_data, y_data, title, xlabel, best_value=None, best_label=None, color="green"):
    ax.barh(y_data, x_data, color=color)
    ax.set_title(title, fontsize=14)
    ax.set_xlabel(xlabel, fontsize=12)
    if best_value is not None and best_label is not None:
        ax.axvline(x=best_value, color="red", linestyle="--", label=best_label)
        ax.legend()
    for i in range(len(x_data)):
        ax.text(x_data.iloc[i], i, f"{x_data.iloc[i]:.6f}", va="center")

In [23]:
def get_metrics_data(filtered_df):
    filtered_df_unique_by_method_name = filtered_df["method_name"].unique()
    analysis_data = []

    for method in filtered_df_unique_by_method_name:
        method_specific_data = filtered_df[filtered_df["method_name"] == method]

        compression_ratio = method_specific_data["compression_ratio_median"].to_numpy()[0]
        duration = method_specific_data["duration_median"].to_numpy()[0]
        total_memory = (
                method_specific_data["ram_mem_used_mb_median"].to_numpy()[0]
                + method_specific_data["gpu_allocated_memory_used_mb_median"].to_numpy()[0]
        )
        frobenius_error = method_specific_data["frobenius_error_median"].to_numpy()[0]

        analysis_data.append(
            {
                "method_name": method,
                "duration_median": duration,
                "total_memory_median": total_memory,
                "frobenius_error_median": frobenius_error,
                "compression_ratio_median": compression_ratio,
            }
        )

    return pd.DataFrame(analysis_data)

In [24]:
# Анализ и визуализация методов по количественным показателям в разрезе качественного показателя
def analyze_data_dfs_best_metrics(df_logs, group_name: str = "decomposition_method"):
    groups = df_logs[group_name].unique()
    analysis_results = {}

    for group in groups:
        filtered_df = df_logs[df_logs[group_name] == group]

        analysis_df = get_metrics_data(filtered_df)

        best_time_method = analysis_df.loc[analysis_df["duration_median"].idxmin()]
        best_memory_method = analysis_df.loc[analysis_df["total_memory_median"].idxmin()]
        best_error_method = analysis_df.loc[analysis_df["frobenius_error_median"].idxmin()]

        analysis_results[group] = {
            "best_time_method": best_time_method.to_dict(),
            "best_memory_method": best_memory_method.to_dict(),
            "best_error_method": best_error_method.to_dict(),
        }

    return analysis_results

In [25]:
# Анализ и визуализация методов по количественным показателям в разрезе качественного показателя
def analyze_data_dfs_worst_metrics(df_logs, group_name: str = "decomposition_method"):
    groups = df_logs[group_name].unique()
    analysis_results = {}

    for group in groups:
        filtered_df = df_logs[df_logs[group_name] == group]

        analysis_df = get_metrics_data(filtered_df)

        best_time_method = analysis_df.loc[analysis_df["duration_median"].idxmax()]
        best_memory_method = analysis_df.loc[analysis_df["total_memory_median"].idxmax()]
        best_error_method = analysis_df.loc[analysis_df["frobenius_error_median"].idxmax()]

        analysis_results[group] = {
            "worst_time_method": best_time_method.to_dict(),
            "worst_memory_method": best_memory_method.to_dict(),
            "worst_error_method": best_error_method.to_dict(),
        }

    return analysis_results

# Расчет лучших метрик

## Сравнение методов по типам данных

In [26]:
analysis_results_data_type = analyze_data_dfs_best_metrics(df_clean_logs, group_name="data_type")

In [27]:
analysis_worst_results_data_type = analyze_data_dfs_worst_metrics(df_clean_logs, group_name="data_type")

## Сравнение метрик по методам

In [28]:
analysis_results_decomposition_method = analyze_data_dfs_best_metrics(df_clean_logs, group_name="decomposition_method")

## Сравнение метрик по методам и типам данных

In [29]:
data_types = df_clean_logs["data_type"].unique()

df_logs_by_data_types_dict = {data_type: df_clean_logs[df_clean_logs["data_type"] == data_type] for data_type in
                              data_types}

analysis_results_data_types_decompositions_methods = {
    data_type: analyze_data_dfs_best_metrics(df_logs_by_data_types_dict[data_type], group_name="decomposition_method")
    for data_type in data_types
}

In [30]:
data_types = df_clean_logs["data_type"].unique()

df_logs_by_data_types_dict_worst = {data_type: df_clean_logs[df_clean_logs["data_type"] == data_type] for data_type in
                              data_types}

analysis_results_data_types_decompositions_methods_worst = {
    data_type: analyze_data_dfs_worst_metrics(df_logs_by_data_types_dict_worst[data_type], group_name="decomposition_method")
    for data_type in data_types
}

# Сравнение метрик

## Метрики в разрезе типов данных

In [31]:
pprint(analysis_results_data_type, indent=4)

{   'eeg-0': {   'best_error_method': {   'compression_ratio_median': 52.237848577500216,
                                          'duration_median': 18.240202498000144,
                                          'frobenius_error_median': 0.023497804068028927,
                                          'method_name': 'TensorLy_TensorTrain_eeg-0_pytorch_randomized_svd',
                                          'total_memory_median': 5759.78515625},
                 'best_memory_method': {   'compression_ratio_median': 52.237848577500216,
                                           'duration_median': 2.4513157450001017,
                                           'frobenius_error_median': 0.02617962018121034,
                                           'method_name': 'TensorLy_TensorTrain_eeg-0_pytorch_truncated_svd',
                                           'total_memory_median': 5404.52197265625},
                 'best_time_method': {   'compression_ratio_median': 52.237848577500216,
 

In [32]:
pprint(analysis_worst_results_data_type, indent=4)

{   'eeg-0': {   'worst_error_method': {   'compression_ratio_median': 51.369509260614535,
                                           'duration_median': 6.294206604999999,
                                           'frobenius_error_median': 1.610640063881874,
                                           'method_name': 'TensorLy_Tucker_eeg-0_pytorch_truncated_svd_random',
                                           'total_memory_median': 9475.07666015625},
                 'worst_memory_method': {   'compression_ratio_median': 51.369509260614535,
                                            'duration_median': 6.294206604999999,
                                            'frobenius_error_median': 1.610640063881874,
                                            'method_name': 'TensorLy_Tucker_eeg-0_pytorch_truncated_svd_random',
                                            'total_memory_median': 9475.07666015625},
                 'worst_time_method': {   'compression_ratio_median': 51.36950926

## Метрики в разрезе методов

In [33]:
pprint(analysis_results_decomposition_method, indent=4)

{   'CP': {   'best_error_method': {   'compression_ratio_median': 50.06507925904784,
                                       'duration_median': 0.8362929030008672,
                                       'frobenius_error_median': 0.06767631857655942,
                                       'method_name': 'TensorLy_CP_image-2_pytorch_truncated_svd_random_False_True_abs_rec_error_1.0_1e-08',
                                       'total_memory_median': 7238.0615234375},
              'best_memory_method': {   'compression_ratio_median': 50.020879006159305,
                                        'duration_median': 1.1333491929690354,
                                        'frobenius_error_median': 0.8004575036466122,
                                        'method_name': 'TensorLy_CP_image-1_pytorch_truncated_svd_random_False_True_rec_error_0.01_1e-05',
                                        'total_memory_median': 3602.08349609375},
              'best_time_method': {   'compression_rati

## Метрики в разрезе типов данных и методов

In [34]:
for data_type, metrics in analysis_results_data_types_decompositions_methods.items():
    print(f"\n{data_type}\n")
    pprint(metrics, indent=4)


image-0

{   'CP': {   'best_error_method': {   'compression_ratio_median': 50.05868249417367,
                                       'duration_median': 0.8219312719884329,
                                       'frobenius_error_median': 0.3219344187527895,
                                       'method_name': 'TensorLy_CP_image-0_pytorch_randomized_svd_random_True_False_rec_error_0.001_1e-09',
                                       'total_memory_median': 4992.32763671875},
              'best_memory_method': {   'compression_ratio_median': 50.05868249417367,
                                        'duration_median': 0.5388988000049721,
                                        'frobenius_error_median': 0.4365097265690565,
                                        'method_name': 'TensorLy_CP_image-0_pytorch_truncated_svd_random_False_False_abs_rec_error_0_1e-08',
                                        'total_memory_median': 4157.3779296875},
              'best_time_method': {   'compres

In [35]:
for data_type, metrics in analysis_results_data_types_decompositions_methods_worst.items():
    print(f"\n{data_type}\n")
    pprint(metrics, indent=4)


image-0

{   'CP': {   'worst_error_method': {   'compression_ratio_median': 50.05868249417367,
                                        'duration_median': 0.09860344001208432,
                                        'frobenius_error_median': 13.67434561252594,
                                        'method_name': 'TensorLy_CP_image-0_pytorch_truncated_svd_random_True_True_rec_error_1.0_1e-06',
                                        'total_memory_median': 4799.37841796875},
              'worst_memory_method': {   'compression_ratio_median': 50.05868249417367,
                                         'duration_median': 1.5522369119998984,
                                         'frobenius_error_median': 4.176430404186249,
                                         'method_name': 'TensorLy_CP_image-0_pytorch_truncated_svd_random_True_False_abs_rec_error_0.5_1e-05',
                                         'total_memory_median': 7426.19677734375},
              'worst_time_method': {   

# Аналитика логов количественных и качественных метрик

## Обработка данных

In [36]:
df_clean_logs.shape

(4772, 30)

In [37]:
df_clean_logs.head()

,method_name,decomposition_method,data_type,language,library,tensor_type,platform,backend,run_type,original_run_indices,...,duration_mean,duration_median,frobenius_error_min,frobenius_error_max,frobenius_error_mean,frobenius_error_median,compression_ratio_min,compression_ratio_max,compression_ratio_mean,compression_ratio_median
0,TensorLy_Tucker_image-0_pytorch_truncated_svd_svd,Tucker,image-0,Python,TensorLy,Dense,"CPU, GPU",pytorch,clean,"[0, 1, 2, 3, 4]",...,2.307812,1.774875,0.672185,0.672185,0.672185,0.672185,50.166931,50.166931,50.166931,50.166931
1,TensorLy_Tucker_image-0_pytorch_truncated_svd_...,Tucker,image-0,Python,TensorLy,Dense,"CPU, GPU",pytorch,clean,"[0, 1, 2, 3, 4]",...,2.658001,2.638634,0.672281,0.672281,0.672281,0.672281,50.166931,50.166931,50.166931,50.166931
2,TensorLy_Tucker_image-0_pytorch_symeig_svd_random,Tucker,image-0,Python,TensorLy,Dense,"CPU, GPU",pytorch,clean,"[0, 1, 2, 3, 4]",...,2.753213,2.702102,0.672281,0.672281,0.672281,0.672281,50.166931,50.166931,50.166931,50.166931
3,TensorLy_Tucker_image-0_pytorch_randomized_svd...,Tucker,image-0,Python,TensorLy,Dense,"CPU, GPU",pytorch,clean,"[0, 1, 2, 3, 4]",...,1.373213,1.388822,0.672679,0.672679,0.672679,0.672679,50.166931,50.166931,50.166931,50.166931
4,TensorLy_Tucker_image-0_pytorch_randomized_svd...,Tucker,image-0,Python,TensorLy,Dense,"CPU, GPU",pytorch,clean,"[0, 1, 2, 3, 4]",...,2.717210,2.757429,0.672281,0.672281,0.672281,0.672281,50.166931,50.166931,50.166931,50.166931


In [38]:
df_clean_logs.columns

Index(['method_name', 'decomposition_method', 'data_type', 'language',
       'library', 'tensor_type', 'platform', 'backend', 'run_type',
       'original_run_indices', 'gpu_allocated_memory_used_mb_min',
       'gpu_allocated_memory_used_mb_max', 'gpu_allocated_memory_used_mb_mean',
       'gpu_allocated_memory_used_mb_median', 'ram_mem_used_mb_min',
       'ram_mem_used_mb_max', 'ram_mem_used_mb_mean', 'ram_mem_used_mb_median',
       'duration_min', 'duration_max', 'duration_mean', 'duration_median',
       'frobenius_error_min', 'frobenius_error_max', 'frobenius_error_mean',
       'frobenius_error_median', 'compression_ratio_min',
       'compression_ratio_max', 'compression_ratio_mean',
       'compression_ratio_median'],
      dtype='object')

In [39]:
df_filtered_for_analytics = df_clean_logs[
    [
        "method_name",
        "decomposition_method",
        "data_type",
        "backend",

        "gpu_allocated_memory_used_mb_min",
        "gpu_allocated_memory_used_mb_median",
        "gpu_allocated_memory_used_mb_mean",
        "gpu_allocated_memory_used_mb_max",

        "ram_mem_used_mb_min",
        "ram_mem_used_mb_median",
        "ram_mem_used_mb_mean",
        "ram_mem_used_mb_max",

        "duration_min",
        "duration_median",
        "duration_mean",
        "duration_max",

        "frobenius_error_min",
        "frobenius_error_median",
        "frobenius_error_mean",
        "frobenius_error_max",

        "compression_ratio_min",
        "compression_ratio_median",
        "compression_ratio_mean",
        "compression_ratio_max",
    ]
]


In [40]:
dfs_by_decomposition_method = {
    decomposition_method: df_filtered_for_analytics[
        df_filtered_for_analytics["decomposition_method"] == decomposition_method
        ]
    for decomposition_method in df_filtered_for_analytics["decomposition_method"].unique()
}

In [41]:
for decomposition_method, df_by_decomposition_method in dfs_by_decomposition_method.items():
    enriched_rows = []
    for _, row in df_by_decomposition_method.iterrows():
        matching_log = next((log for log in logs if row["method_name"] == log["method_name"]), None)

        enriched_row = row.copy()

        if row["decomposition_method"] == "Tucker":
            enriched_row["init"] = matching_log["method_args"].get("init")
            enriched_row["svd"] = matching_log["method_args"].get("svd")
        elif row["decomposition_method"] == "TensorTrain":
            enriched_row["svd"] = matching_log["method_args"].get("svd")
        elif row["decomposition_method"] == "CP":
            enriched_row["init"] = matching_log["method_args"].get("init")
            enriched_row["svd"] = matching_log["method_args"].get("svd")
            enriched_row["normalize_factors"] = matching_log["method_args"].get("normalize_factors")
            enriched_row["orthogonalise"] = matching_log["method_args"].get("orthogonalise")
            enriched_row["tol"] = matching_log["method_args"].get("tol")
            enriched_row["l2_reg"] = matching_log["method_args"].get("l2_reg")
            enriched_row["cvg_criterion"] = matching_log["method_args"].get("cvg_criterion")

        enriched_rows.append(enriched_row)

    dfs_by_decomposition_method[decomposition_method] = pd.DataFrame(enriched_rows)

In [42]:
dfs_by_decomposition_method.keys()

dict_keys(['Tucker', 'TensorTrain', 'CP'])

## Обработка ошибок

In [43]:
df_logs_errors_without_singular_matrix.shape

(9828, 11)

In [44]:
df_logs_errors_without_singular_matrix.head()

,method_name,decomposition_method,data_type,language,library,tensor_type,platform,backend,error_message,is_error,memory_tried_to_allocated_error_mb
0,TensorLy_Tucker_image-0_pytorch_truncated_svd_svd,Tucker,image-0,Python,TensorLy,Dense,"CPU, GPU",pytorch,,False,0.0
1,TensorLy_Tucker_image-0_pytorch_truncated_svd_...,Tucker,image-0,Python,TensorLy,Dense,"CPU, GPU",pytorch,,False,0.0
2,TensorLy_Tucker_image-0_pytorch_symeig_svd_svd,Tucker,image-0,Python,TensorLy,Dense,"CPU, GPU",pytorch,"Пропущена итерация из-за ошибки: pytorch, syme...",True,385996.8
3,TensorLy_Tucker_image-0_pytorch_symeig_svd_random,Tucker,image-0,Python,TensorLy,Dense,"CPU, GPU",pytorch,,False,0.0
4,TensorLy_Tucker_image-0_pytorch_randomized_svd...,Tucker,image-0,Python,TensorLy,Dense,"CPU, GPU",pytorch,,False,0.0


In [45]:
df_logs_errors_without_singular_matrix.columns

Index(['method_name', 'decomposition_method', 'data_type', 'language',
       'library', 'tensor_type', 'platform', 'backend', 'error_message',
       'is_error', 'memory_tried_to_allocated_error_mb'],
      dtype='object')

In [46]:
df_filtered_errors_for_analytics = df_logs_errors_without_singular_matrix[
    [
        "method_name",
        "decomposition_method",
        "data_type",
        # "backend",

        "memory_tried_to_allocated_error_mb",
        "is_error"
    ]
]


In [47]:
dfs_errors_by_decomposition_method = {
    decomposition_method: df_filtered_errors_for_analytics[
        df_filtered_errors_for_analytics["decomposition_method"] == decomposition_method
        ]
    for decomposition_method in df_filtered_errors_for_analytics["decomposition_method"].unique()
}

for decomposition_method, df_by_decomposition_method in dfs_errors_by_decomposition_method.items():
    enriched_rows = []
    for _, row in df_by_decomposition_method.iterrows():
        matching_log = next((log for log in logs if row["method_name"] == log["method_name"]), None)

        enriched_row = row.copy()

        if row["decomposition_method"] == "Tucker":
            enriched_row["init"] = matching_log["method_args"].get("init")
            enriched_row["svd"] = matching_log["method_args"].get("svd")

            if enriched_row["svd"] is None or enriched_row["init"] is None:
                print(matching_log["method_args"])
        elif row["decomposition_method"] == "TensorTrain":
            if "T3F" in row["method_name"]:
                enriched_row["svd"] = "T3F implementation"
            else:
                enriched_row["svd"] = matching_log["method_args"].get("svd")

            if enriched_row["svd"] is None:
                print(matching_log["method_args"])
        elif row["decomposition_method"] == "CP":
            enriched_row["init"] = matching_log["method_args"].get("init")
            enriched_row["svd"] = matching_log["method_args"].get("svd")
            enriched_row["normalize_factors"] = matching_log["method_args"].get("normalize_factors")
            enriched_row["orthogonalise"] = matching_log["method_args"].get("orthogonalise")
            enriched_row["tol"] = matching_log["method_args"].get("tol")
            enriched_row["l2_reg"] = matching_log["method_args"].get("l2_reg")
            enriched_row["cvg_criterion"] = matching_log["method_args"].get("cvg_criterion")

            if (
                    enriched_row["svd"] is None or
                    enriched_row["init"] is None or
                    enriched_row["normalize_factors"] is None or
                    enriched_row["orthogonalise"] is None or
                    enriched_row["tol"] is None or
                    enriched_row["l2_reg"] is None or
                    enriched_row["cvg_criterion"] is None
            ):
                print(matching_log["method_args"])

        enriched_rows.append(enriched_row)

    dfs_errors_by_decomposition_method[decomposition_method] = pd.DataFrame(enriched_rows)

In [48]:
dfs_errors_by_decomposition_method.keys()

dict_keys(['Tucker', 'TensorTrain', 'CP'])

## Методы для графиков

In [49]:
def preprocess_dataframe(df):
    df_encoded = df.copy()
    for column in df_encoded.columns:
        if df_encoded[column].dtype == "object" and column != "method_name":
            df_encoded[column] = LabelEncoder().fit_transform(df_encoded[column])

    scaler = MinMaxScaler()
    df_encoded[df_encoded.columns.difference(["method_name"])] = scaler.fit_transform(
        df_encoded[df_encoded.columns.difference(["method_name"])]
    )
    return df_encoded

In [50]:
def plot_heatmap(df, title):
    folder_path = Path("../.cache/data_analyze/")

    corr_matrix = df.drop(columns=["method_name"]).corr()

    fig = go.Figure(
        data=go.Heatmap(
            z=corr_matrix.values,
            x=corr_matrix.columns,
            y=corr_matrix.index,
            colorscale=colors.sequential.RdBu,
            colorbar={
                "title": "Correlation",
                "titlefont": {"color": "black"},
                "tickfont": {"color": "black"},
            },
            text=corr_matrix.values,
            texttemplate="%{text:.2f}",
            textfont={"size": 12, "color": "black"},
            hovertemplate="<b>%{x}</b> vs. <b>%{y}</b> <b>Correlation:</b> %{z:.8f}<extra></extra>",
        )
    )

    fig.update_layout(
        title=f"Heatmap: {title}",
        title_font={"color": "black"},
        xaxis={"tickangle": -45, "tickfont": {"size": 12, "color": "black"}},
        yaxis={"tickfont": {"size": 12, "color": "black"}},
        width=1400,
        height=1400,
        plot_bgcolor="white",
        paper_bgcolor="white",
    )

    folder_path.mkdir(parents=True, exist_ok=True)

    fig.write_html(folder_path / f"heatmap_{title}.html")

In [51]:
def plot_pca(df, title):
    folder_path = Path("../.cache/data_analyze/")
    folder_path.mkdir(parents=True, exist_ok=True)

    pca_df = df.drop(columns=["method_name"])
    pca = PCA(n_components=3)
    pca_components = pca.fit_transform(pca_df)

    pca_df = pd.DataFrame(
        {
            "PCA1": pca_components[:, 0],
            "PCA2": pca_components[:, 1],
            "PCA3": pca_components[:, 2],
            "method_name": df["method_name"].to_numpy(),
        }
    )

    fig1 = px.scatter(
        pca_df, x="PCA1", y="PCA2", color="method_name", title=f"PCA 2D Projection: {title}", hover_name="method_name"
    )
    fig1.update_traces(marker={"size": 10})
    fig1.update_layout(
        width=1800,
        height=1400,
        plot_bgcolor="white",
        paper_bgcolor="white",
        title_font={"color": "black"},
        xaxis={
            "titlefont": {"size": 12, "color": "black"},
            "tickfont": {"size": 12, "color": "black"},
        },
        yaxis={
            "titlefont": {"size": 12, "color": "black"},
            "tickfont": {"size": 12, "color": "black"},
        },
        showlegend=False,
    )

    fig1.write_html(folder_path / f"pca_2d_{title}.html")

    fig2 = px.scatter_3d(
        pca_df,
        x="PCA1",
        y="PCA2",
        z="PCA3",
        color="method_name",
        title=f"PCA 3D Projection: {title}",
        hover_name="method_name",
    )
    fig2.update_traces(marker={"size": 5})
    fig2.update_layout(
        scene={
            "xaxis_title": "PCA1",
            "yaxis_title": "PCA2",
            "zaxis_title": "PCA3",
            "xaxis_backgroundcolor": "white",
            "yaxis_backgroundcolor": "white",
            "zaxis_backgroundcolor": "white",
            "xaxis": {
                "titlefont": {"size": 12, "color": "black"},
                "tickfont": {"size": 12, "color": "black"},
            },
            "yaxis": {
                "titlefont": {"size": 12, "color": "black"},
                "tickfont": {"size": 12, "color": "black"},
            },
            "zaxis": {
                "titlefont": {"size": 12, "color": "black"},
                "tickfont": {"size": 12, "color": "black"},
            },
            "bgcolor": "white",
        },
        width=1800,
        height=1400,
        plot_bgcolor="white",
        paper_bgcolor="white",
        title_font={"color": "black"},
        showlegend=False,
    )

    fig2.write_html(folder_path / f"pca_3d_{title}.html")

In [52]:
def plot_pairplot(df, title, sample_size=1000, num_axes=100):
    folder_path = Path("../.cache/data_analyze/")
    folder_path.mkdir(parents=True, exist_ok=True)

    pairplot_df = df.drop(columns=["method_name"])

    if len(pairplot_df) > sample_size:
        pairplot_df = pairplot_df.sample(n=sample_size, random_state=42)

    selected_axes = pairplot_df.columns[:num_axes]

    sns.set(style="white")
    g = sns.pairplot(pairplot_df[selected_axes], diag_kind="kde", markers="+", plot_kws={"s": 50})
    g.fig.suptitle(f"Pairplot: {title}", y=1.02, fontsize=16, color="black")

    for ax in g.axes.flatten():
        ax.tick_params(axis="x", labelsize=12, labelcolor="black")
        ax.tick_params(axis="y", labelsize=12, labelcolor="black")
        ax.set_facecolor("white")
        ax.set_xticks(ax.get_xticks())
        ax.set_xticklabels(ax.get_xticklabels(), rotation=45, ha="right")

    plt.savefig(folder_path / f"pairplot_{title}.png", dpi=300, bbox_inches="tight")
    plt.close("all")

## Энкодинг и нормализация датафреймов

In [53]:
dfs_by_decomposition_method_encoded = {}

for key, df_by_decomposition_method in dfs_by_decomposition_method.items():
    dfs_by_decomposition_method_encoded[key] = preprocess_dataframe(
        df_by_decomposition_method.drop("decomposition_method", axis=1)
    )

In [54]:
dfs_errors_by_decomposition_method_encoded = {}

for key, df_by_decomposition_method in dfs_errors_by_decomposition_method.items():
    dfs_errors_by_decomposition_method_encoded[key] = preprocess_dataframe(
        df_by_decomposition_method.drop("decomposition_method", axis=1)
    )

# Графики для данных

## Heatmap

### по методам декомпозиции

In [55]:
method_name = "Tucker_data"
plot_heatmap(dfs_by_decomposition_method_encoded[method_name.split("_")[0]], title=method_name)

method_name = "TensorTrain_data"
plot_heatmap(dfs_by_decomposition_method_encoded[method_name.split("_")[0]], title=method_name)

method_name = "CP_data"
plot_heatmap(dfs_by_decomposition_method_encoded[method_name.split("_")[0]], title=method_name)

### Heatmaps по разным типам тензоров и отдельным тензорам

#### Tucker

##### images

In [56]:
df_tucker_image_0 = dfs_by_decomposition_method['Tucker'].loc[dfs_by_decomposition_method['Tucker']['data_type'] == 'image-0']
df_tucker_image_0_encoded = preprocess_dataframe(
        df_tucker_image_0.drop("decomposition_method", axis=1)
)
plot_heatmap(df_tucker_image_0_encoded, title="Tucker_data_image-0")


df_tucker_image_1 = dfs_by_decomposition_method['Tucker'].loc[dfs_by_decomposition_method['Tucker']['data_type'] == 'image-1']
df_tucker_image_1_encoded = preprocess_dataframe(
        df_tucker_image_1.drop("decomposition_method", axis=1)
)
plot_heatmap(df_tucker_image_1_encoded, title="Tucker_data_image-1")


df_tucker_image_2 = dfs_by_decomposition_method['Tucker'].loc[dfs_by_decomposition_method['Tucker']['data_type'] == 'image-2']
df_tucker_image_2_encoded = preprocess_dataframe(
        df_tucker_image_2.drop("decomposition_method", axis=1)
)
plot_heatmap(df_tucker_image_2_encoded, title="Tucker_data_image-2")


mask = dfs_by_decomposition_method['Tucker']['data_type'].isin(['image-0', 'image-1', 'image-2'])
df_tucker_images = dfs_by_decomposition_method['Tucker'].loc[mask]
df_tucker_images_encoded = preprocess_dataframe(
        df_tucker_images.drop("decomposition_method", axis=1)
)
plot_heatmap(df_tucker_images_encoded, title="Tucker_data_images")

##### videos

In [57]:
df_tucker_video_0 = dfs_by_decomposition_method['Tucker'].loc[dfs_by_decomposition_method['Tucker']['data_type'] == 'video-0']
df_tucker_video_0_encoded = preprocess_dataframe(
        df_tucker_video_0.drop("decomposition_method", axis=1)
)
plot_heatmap(df_tucker_video_0_encoded, title="Tucker_data_video-0")


df_tucker_video_1 = dfs_by_decomposition_method['Tucker'].loc[dfs_by_decomposition_method['Tucker']['data_type'] == 'video-1']
df_tucker_video_1_encoded = preprocess_dataframe(
        df_tucker_video_1.drop("decomposition_method", axis=1)
)
plot_heatmap(df_tucker_video_1_encoded, title="Tucker_data_video-1")


df_tucker_video_2 = dfs_by_decomposition_method['Tucker'].loc[dfs_by_decomposition_method['Tucker']['data_type'] == 'video-2']
df_tucker_video_2_encoded = preprocess_dataframe(
        df_tucker_video_2.drop("decomposition_method", axis=1)
)
plot_heatmap(df_tucker_video_2_encoded, title="Tucker_data_video-2")


mask = dfs_by_decomposition_method['Tucker']['data_type'].isin(['video-0', 'video-1', 'video-2'])
df_tucker_videos = dfs_by_decomposition_method['Tucker'].loc[mask]
df_tucker_videos_encoded = preprocess_dataframe(
        df_tucker_videos.drop("decomposition_method", axis=1)
)
plot_heatmap(df_tucker_videos_encoded, title="Tucker_data_videos")

##### EEGs

In [58]:
df_tucker_eeg_0 = dfs_by_decomposition_method['Tucker'].loc[dfs_by_decomposition_method['Tucker']['data_type'] == 'eeg-0']
df_tucker_eeg_0_encoded = preprocess_dataframe(
        df_tucker_eeg_0.drop("decomposition_method", axis=1)
)
plot_heatmap(df_tucker_eeg_0_encoded, title="Tucker_data_eeg-0")


df_tucker_eeg_1 = dfs_by_decomposition_method['Tucker'].loc[dfs_by_decomposition_method['Tucker']['data_type'] == 'eeg-1']
df_tucker_eeg_1_encoded = preprocess_dataframe(
        df_tucker_eeg_1.drop("decomposition_method", axis=1)
)
plot_heatmap(df_tucker_eeg_1_encoded, title="Tucker_data_eeg-1")


mask = dfs_by_decomposition_method['Tucker']['data_type'].isin(['eeg-0', 'eeg-1'])
df_tucker_eegs = dfs_by_decomposition_method['Tucker'].loc[mask]
df_tucker_eegs_encoded = preprocess_dataframe(
        df_tucker_eegs.drop("decomposition_method", axis=1)
)
plot_heatmap(df_tucker_eegs_encoded, title="Tucker_data_eegs")

#### CP

##### images

In [59]:
df_CP_image_0 = dfs_by_decomposition_method['CP'].loc[dfs_by_decomposition_method['CP']['data_type'] == 'image-0']
df_CP_image_0_encoded = preprocess_dataframe(
        df_CP_image_0.drop("decomposition_method", axis=1)
)
plot_heatmap(df_CP_image_0_encoded, title="CP_data_image-0")


df_CP_image_1 = dfs_by_decomposition_method['CP'].loc[dfs_by_decomposition_method['CP']['data_type'] == 'image-1']
df_CP_image_1_encoded = preprocess_dataframe(
        df_CP_image_1.drop("decomposition_method", axis=1)
)
plot_heatmap(df_CP_image_1_encoded, title="CP_data_image-1")


df_CP_image_2 = dfs_by_decomposition_method['CP'].loc[dfs_by_decomposition_method['CP']['data_type'] == 'image-2']
df_CP_image_2_encoded = preprocess_dataframe(
        df_CP_image_2.drop("decomposition_method", axis=1)
)
plot_heatmap(df_CP_image_2_encoded, title="CP_data_image-2")


mask = dfs_by_decomposition_method['CP']['data_type'].isin(['image-0', 'image-1', 'image-2'])
df_CP_images = dfs_by_decomposition_method['CP'].loc[mask]
df_CP_images_encoded = preprocess_dataframe(
        df_CP_images.drop("decomposition_method", axis=1)
)
plot_heatmap(df_CP_images_encoded, title="CP_data_images")

#### TensorTrain

##### images

In [60]:
df_tensor_train_image_0 = dfs_by_decomposition_method['TensorTrain'].loc[dfs_by_decomposition_method['TensorTrain']['data_type'] == 'image-0']
df_tensor_train_image_0_encoded = preprocess_dataframe(
        df_tensor_train_image_0.drop("decomposition_method", axis=1)
)
plot_heatmap(df_tensor_train_image_0_encoded, title="TensorTrain_data_image-0")


df_tensor_train_image_1 = dfs_by_decomposition_method['TensorTrain'].loc[dfs_by_decomposition_method['TensorTrain']['data_type'] == 'image-1']
df_tensor_train_image_1_encoded = preprocess_dataframe(
        df_tensor_train_image_1.drop("decomposition_method", axis=1)
)
plot_heatmap(df_tensor_train_image_1_encoded, title="TensorTrain_data_image-1")


df_tensor_train_image_2 = dfs_by_decomposition_method['TensorTrain'].loc[dfs_by_decomposition_method['TensorTrain']['data_type'] == 'image-2']
df_tensor_train_image_2_encoded = preprocess_dataframe(
        df_tensor_train_image_2.drop("decomposition_method", axis=1)
)
plot_heatmap(df_tensor_train_image_2_encoded, title="TensorTrain_data_image-2")


mask = dfs_by_decomposition_method['TensorTrain']['data_type'].isin(['image-0', 'image-1', 'image-2'])
df_tensor_train_images = dfs_by_decomposition_method['TensorTrain'].loc[mask]
df_tensor_train_images_encoded = preprocess_dataframe(
        df_tensor_train_images.drop("decomposition_method", axis=1)
)
plot_heatmap(df_tensor_train_images_encoded, title="TensorTrain_data_images")

##### videos

In [61]:
df_tensor_train_video_0 = dfs_by_decomposition_method['TensorTrain'].loc[dfs_by_decomposition_method['TensorTrain']['data_type'] == 'video-0']
df_tensor_train_video_0_encoded = preprocess_dataframe(
        df_tensor_train_video_0.drop("decomposition_method", axis=1)
)
plot_heatmap(df_tensor_train_video_0_encoded, title="TensorTrain_data_video-0")


df_tensor_train_video_1 = dfs_by_decomposition_method['TensorTrain'].loc[dfs_by_decomposition_method['TensorTrain']['data_type'] == 'video-1']
df_tensor_train_video_1_encoded = preprocess_dataframe(
        df_tensor_train_video_1.drop("decomposition_method", axis=1)
)
plot_heatmap(df_tensor_train_video_1_encoded, title="TensorTrain_data_video-1")


df_tensor_train_video_2 = dfs_by_decomposition_method['TensorTrain'].loc[dfs_by_decomposition_method['TensorTrain']['data_type'] == 'video-2']
df_tensor_train_video_2_encoded = preprocess_dataframe(
        df_tensor_train_video_2.drop("decomposition_method", axis=1)
)
plot_heatmap(df_tensor_train_video_2_encoded, title="TensorTrain_data_video-2")


mask = dfs_by_decomposition_method['TensorTrain']['data_type'].isin(['video-0', 'video-1', 'video-2'])
df_tensor_train_videos = dfs_by_decomposition_method['TensorTrain'].loc[mask]
df_tensor_train_videos_encoded = preprocess_dataframe(
        df_tensor_train_videos.drop("decomposition_method", axis=1)
)
plot_heatmap(df_tensor_train_videos_encoded, title="TensorTrain_data_videos")

##### EEGs

In [62]:
df_tensor_train_eeg_0 = dfs_by_decomposition_method['TensorTrain'].loc[dfs_by_decomposition_method['TensorTrain']['data_type'] == 'eeg-0']
df_tensor_train_eeg_0_encoded = preprocess_dataframe(
        df_tensor_train_eeg_0.drop("decomposition_method", axis=1)
)
plot_heatmap(df_tensor_train_eeg_0_encoded, title="TensorTrain_data_eeg-0")


df_tensor_train_eeg_1 = dfs_by_decomposition_method['TensorTrain'].loc[dfs_by_decomposition_method['TensorTrain']['data_type'] == 'eeg-1']
df_tensor_train_eeg_1_encoded = preprocess_dataframe(
        df_tensor_train_eeg_1.drop("decomposition_method", axis=1)
)
plot_heatmap(df_tensor_train_eeg_1_encoded, title="TensorTrain_data_eeg-1")


mask = dfs_by_decomposition_method['TensorTrain']['data_type'].isin(['eeg-0', 'eeg-1'])
df_tensor_train_eegs = dfs_by_decomposition_method['TensorTrain'].loc[mask]
df_tensor_train_eegs_encoded = preprocess_dataframe(
        df_tensor_train_eegs.drop("decomposition_method", axis=1)
)
plot_heatmap(df_tensor_train_eegs_encoded, title="TensorTrain_data_eegs")

## PCA

In [63]:
method_name = "Tucker_data"
plot_pca(dfs_by_decomposition_method_encoded[method_name.split("_")[0]], title=method_name)

method_name = "TensorTrain_data"
plot_pca(dfs_by_decomposition_method_encoded[method_name.split("_")[0]], title=method_name)

method_name = "CP_data"
plot_pca(dfs_by_decomposition_method_encoded[method_name.split("_")[0]], title=method_name)

## Pairplot

In [64]:
method_name = "Tucker_data"
plot_pairplot(dfs_by_decomposition_method_encoded[method_name.split("_")[0]], title=method_name)
None

method_name = "TensorTrain_data"
plot_pairplot(dfs_by_decomposition_method_encoded[method_name.split("_")[0]], title=method_name)
None

method_name = "CP_data"
plot_pairplot(dfs_by_decomposition_method_encoded[method_name.split("_")[0]], title=method_name)
None

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

# Графики для ошибок

## Heatmap

In [65]:
method_name = "Tucker_errors"
plot_heatmap(dfs_errors_by_decomposition_method_encoded[method_name.split("_")[0]], title=method_name)

method_name = "TensorTrain_errors"
plot_heatmap(dfs_errors_by_decomposition_method_encoded[method_name.split("_")[0]], title=method_name)

method_name = "CP_errors"
plot_heatmap(dfs_errors_by_decomposition_method_encoded[method_name.split("_")[0]], title=method_name)

## PCA

In [66]:
method_name = "Tucker_errors"
plot_pca(dfs_errors_by_decomposition_method_encoded[method_name.split("_")[0]], title=method_name)

method_name = "TensorTrain_errors"
plot_pca(dfs_errors_by_decomposition_method_encoded[method_name.split("_")[0]], title=method_name)

method_name = "CP_errors"
plot_pca(dfs_errors_by_decomposition_method_encoded[method_name.split("_")[0]], title=method_name)

## Pairplot

In [67]:
method_name = "Tucker_errors"
plot_pairplot(dfs_errors_by_decomposition_method_encoded[method_name.split("_")[0]], title=method_name)
None

method_name = "TensorTrain_errors"
plot_pairplot(dfs_errors_by_decomposition_method_encoded[method_name.split("_")[0]], title=method_name)
None

method_name = "CP_errors"
plot_pairplot(dfs_errors_by_decomposition_method_encoded[method_name.split("_")[0]], title=method_name)
None

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

# Анализ на основе выводов из графиков

In [68]:
def get_n_rows_near_quantile(df, column, quantile_value, n=10):
    diffs = (df[column] - quantile_value).abs()
    nearest_indices = diffs.nsmallest(n).index
    return df.loc[nearest_indices].copy()

## ошибки

In [69]:
dfs_errors_by_decomposition_method.keys()

dict_keys(['Tucker', 'TensorTrain', 'CP'])

In [70]:
dfs_errors_by_decomposition_method_encoded.keys()

dict_keys(['Tucker', 'TensorTrain', 'CP'])

### CP

In [71]:
# квантили по памяти
mask = (
        (
                dfs_errors_by_decomposition_method['CP']['is_error'] == True
        )
        &
        (
                dfs_errors_by_decomposition_method['CP']['memory_tried_to_allocated_error_mb'] > 0.0
        )
)
errors_cp = dfs_errors_by_decomposition_method['CP'].loc[
    mask
]

errors_cp_sorted = errors_cp.sort_values(by='memory_tried_to_allocated_error_mb')

quantiles = [0.0, 0.5, 0.95, 0.98, 1.0]
quantile_values = errors_cp_sorted['memory_tried_to_allocated_error_mb'].quantile(quantiles)

rows_by_quantiles = {
    f"{int(q * 100)}%": get_n_rows_near_quantile(errors_cp_sorted, 'memory_tried_to_allocated_error_mb', quantile_values[q], n=10)
    for q in quantiles
}
df_logs_cp_errors_rows_near_all_quantiles = pd.concat(rows_by_quantiles.values(), keys=rows_by_quantiles.keys())

In [72]:
df_logs_cp_errors_rows_near_all_quantiles

method_name  \
0%   9821  TensorLy_CP_image-1_pytorch_randomized_svd_svd...   
     7343  TensorLy_CP_image-1_pytorch_randomized_svd_svd...   
     7347  TensorLy_CP_image-1_pytorch_truncated_svd_svd_...   
     7348  TensorLy_CP_image-1_pytorch_symeig_svd_svd_Fal...   
     7349  TensorLy_CP_image-1_pytorch_randomized_svd_svd...   
     7353  TensorLy_CP_image-1_pytorch_truncated_svd_svd_...   
     7354  TensorLy_CP_image-1_pytorch_symeig_svd_svd_Fal...   
     7355  TensorLy_CP_image-1_pytorch_randomized_svd_svd...   
     7359  TensorLy_CP_image-1_pytorch_truncated_svd_svd_...   
     7360  TensorLy_CP_image-1_pytorch_symeig_svd_svd_Fal...   
50%  5366  TensorLy_CP_image-0_pytorch_truncated_svd_svd_...   
     5612  TensorLy_CP_image-0_pytorch_symeig_svd_svd_Tru...   
     5613  TensorLy_CP_image-0_pytorch_randomized_svd_svd...   
     5617  TensorLy_CP_image-0_pytorch_truncated_svd_svd_...   
     5618  TensorLy_CP_image-0_pytorch_symeig_svd_svd_Tru...   
     5619  TensorLy_CP_image-0_pytorch_randomized_svd_svd...   
     5623  TensorLy_CP_image-0_pytorch_truncated_svd_svd_...   
     5624  TensorLy_CP_image-0_pytorch_symeig_svd_svd_Tru...   
     5625  TensorLy_CP_image-0_pytorch_randomized_svd_svd...   
     5629  TensorLy_CP_image-0_pytorch_truncated_svd_svd_...   
95%  1152  TensorLy_CP_image-2_pytorch_truncated_svd_svd_...   
     1135  TensorLy_CP_image-2_pytorch_symeig_svd_svd_Fal...   
     1136  TensorLy_CP_image-2_pytorch_randomized_svd_svd...   
     1140  TensorLy_CP_image-2_pytorch_truncated_svd_svd_...   
     1141  TensorLy_CP_image-2_pytorch_symeig_svd_svd_Fal...   
     1142  TensorLy_CP_image-2_pytorch_randomized_svd_svd...   
     1146  TensorLy_CP_image-2_pytorch_truncated_svd_svd_...   
     1147  TensorLy_CP_image-2_pytorch_symeig_svd_svd_Fal...   
     1148  TensorLy_CP_image-2_pytorch_randomized_svd_svd...   
     1153  TensorLy_CP_image-2_pytorch_symeig_svd_svd_Fal...   
98%  1152  TensorLy_CP_image-2_pytorch_truncated_svd_svd_...   
     1135  TensorLy_CP_image-2_pytorch_symeig_svd_svd_Fal...   
     1136  TensorLy_CP_image-2_pytorch_randomized_svd_svd...   
     1140  TensorLy_CP_image-2_pytorch_truncated_svd_svd_...   
     1141  TensorLy_CP_image-2_pytorch_symeig_svd_svd_Fal...   
     1142  TensorLy_CP_image-2_pytorch_randomized_svd_svd...   
     1146  TensorLy_CP_image-2_pytorch_truncated_svd_svd_...   
     1147  TensorLy_CP_image-2_pytorch_symeig_svd_svd_Fal...   
     1148  TensorLy_CP_image-2_pytorch_randomized_svd_svd...   
     1153  TensorLy_CP_image-2_pytorch_symeig_svd_svd_Fal...   
100% 1152  TensorLy_CP_image-2_pytorch_truncated_svd_svd_...   
     1135  TensorLy_CP_image-2_pytorch_symeig_svd_svd_Fal...   
     1136  TensorLy_CP_image-2_pytorch_randomized_svd_svd...   
     1140  TensorLy_CP_image-2_pytorch_truncated_svd_svd_...   
     1141  TensorLy_CP_image-2_pytorch_symeig_svd_svd_Fal...   
     1142  TensorLy_CP_image-2_pytorch_randomized_svd_svd...   
     1146  TensorLy_CP_image-2_pytorch_truncated_svd_svd_...   
     1147  TensorLy_CP_image-2_pytorch_symeig_svd_svd_Fal...   
     1148  TensorLy_CP_image-2_pytorch_randomized_svd_svd...   
     1153  TensorLy_CP_image-2_pytorch_symeig_svd_svd_Fal...   

          decomposition_method data_type  memory_tried_to_allocated_error_mb  \
0%   9821                   CP   image-1                           248903.68   
     7343                   CP   image-1                           248903.68   
     7347                   CP   image-1                           248903.68   
     7348                   CP   image-1                           248903.68   
     7349                   CP   image-1                           248903.68   
     7353                   CP   image-1                           248903.68   
     7354                   CP   image-1                           248903.68   
     7355                   CP   image-1                           248903.68   
     7359                   CP   image-1                        

In [73]:
dfs_errors_by_decomposition_method['CP'].loc[
    dfs_errors_by_decomposition_method['CP']['is_error'] == False,
    'init'
].value_counts(dropna=False)

init
random    4710
Name: count, dtype: int64

In [74]:
dfs_errors_by_decomposition_method['CP'].loc[
    dfs_errors_by_decomposition_method['CP']['is_error'] == True,
    'init'
].value_counts(dropna=False)

init
svd    5040
Name: count, dtype: int64

значение для init = svd приводит к резкому увеличению требуемой памяти, требуемая память зависит от начального тензора

### Tensor-Train

In [75]:
dfs_errors_by_decomposition_method['TensorTrain'][
    'svd'
].value_counts(dropna=False)

svd
randomized_svd        8
truncated_svd         8
symeig_svd            8
T3F implementation    6
Name: count, dtype: int64

In [76]:
dfs_errors_by_decomposition_method['TensorTrain'].loc[
    dfs_errors_by_decomposition_method['TensorTrain']['is_error'] == False,
    'svd'
].value_counts(dropna=False)

svd
randomized_svd        8
truncated_svd         8
T3F implementation    6
Name: count, dtype: int64

In [77]:
dfs_errors_by_decomposition_method['TensorTrain'].loc[
    dfs_errors_by_decomposition_method['TensorTrain']['is_error'] == True
]

,method_name,decomposition_method,data_type,memory_tried_to_allocated_error_mb,is_error,svd
50,TensorLy_TensorTrain_image-0_pytorch_symeig_svd,TensorTrain,image-0,5.453824e+04,True,symeig_svd
53,TensorLy_TensorTrain_image-1_pytorch_symeig_svd,TensorTrain,image-1,7.960576e+04,True,symeig_svd
56,TensorLy_TensorTrain_image-2_pytorch_symeig_svd,TensorTrain,image-2,4.549837e+05,True,symeig_svd
59,TensorLy_TensorTrain_video-0_pytorch_symeig_svd,TensorTrain,video-0,4.665344e+04,True,symeig_svd
62,TensorLy_TensorTrain_video-1_pytorch_symeig_svd,TensorTrain,video-1,2.624512e+04,True,symeig_svd
65,TensorLy_TensorTrain_video-2_pytorch_symeig_svd,TensorTrain,video-2,4.665344e+04,True,symeig_svd
68,TensorLy_TensorTrain_eeg-0_pytorch_symeig_svd,TensorTrain,eeg-0,3.322946e+09,True,symeig_svd
71,TensorLy_TensorTrain_eeg-1_pytorch_symeig_svd,TensorTrain,eeg-1,1.184237e+10,True,symeig_svd


symeig_svd в tensorly в tucker вызывает ошибку выделения памяти

### Tucker

In [78]:
dfs_errors_by_decomposition_method['Tucker'][
    'init'
].value_counts(dropna=False)

init
svd       24
random    24
Name: count, dtype: int64

In [79]:
dfs_errors_by_decomposition_method['Tucker'][
    'svd'
].value_counts(dropna=False)

svd
truncated_svd     16
symeig_svd        16
randomized_svd    16
Name: count, dtype: int64

In [80]:
dfs_errors_by_decomposition_method['Tucker'].loc[
    dfs_errors_by_decomposition_method['Tucker']['is_error'] == False,
    'init'
].value_counts(dropna=False)

init
random    24
svd       16
Name: count, dtype: int64

In [81]:
dfs_errors_by_decomposition_method['Tucker'].loc[
    dfs_errors_by_decomposition_method['Tucker']['is_error'] == True,
    'init'
].value_counts(dropna=False)

init
svd    8
Name: count, dtype: int64

In [82]:
dfs_errors_by_decomposition_method['Tucker'].loc[
    dfs_errors_by_decomposition_method['Tucker']['is_error'] == True
]

,method_name,decomposition_method,data_type,memory_tried_to_allocated_error_mb,is_error,init,svd
2,TensorLy_Tucker_image-0_pytorch_symeig_svd_svd,Tucker,image-0,3.859968e+05,True,svd,symeig_svd
8,TensorLy_Tucker_image-1_pytorch_symeig_svd_svd,Tucker,image-1,2.489037e+05,True,svd,symeig_svd
14,TensorLy_Tucker_image-2_pytorch_symeig_svd_svd,Tucker,image-2,2.586040e+06,True,svd,symeig_svd
20,TensorLy_Tucker_video-0_pytorch_symeig_svd_svd,Tucker,video-0,4.665344e+04,True,svd,symeig_svd
26,TensorLy_Tucker_video-1_pytorch_symeig_svd_svd,Tucker,video-1,2.624512e+04,True,svd,symeig_svd
32,TensorLy_Tucker_video-2_pytorch_symeig_svd_svd,Tucker,video-2,4.665344e+04,True,svd,symeig_svd
38,TensorLy_Tucker_eeg-0_pytorch_symeig_svd_svd,Tucker,eeg-0,3.322946e+09,True,svd,symeig_svd
44,TensorLy_Tucker_eeg-1_pytorch_symeig_svd_svd,Tucker,eeg-1,1.184237e+10,True,svd,symeig_svd


init = svd и svd = symeig_svd когда ставятся одновременно приводят к взрыву требуемой памяти

In [83]:
mask = (
        (
                dfs_errors_by_decomposition_method['Tucker']['svd'] == 'symeig_svd'
        )
)

dfs_errors_by_decomposition_method['Tucker'].loc[
    mask
].value_counts(dropna=False)

method_name                                        decomposition_method  data_type  memory_tried_to_allocated_error_mb  is_error  init    svd       
TensorLy_Tucker_eeg-0_pytorch_symeig_svd_random    Tucker                eeg-0      0.000000e+00                        False     random  symeig_svd    1
TensorLy_Tucker_eeg-0_pytorch_symeig_svd_svd       Tucker                eeg-0      3.322946e+09                        True      svd     symeig_svd    1
TensorLy_Tucker_eeg-1_pytorch_symeig_svd_random    Tucker                eeg-1      0.000000e+00                        False     random  symeig_svd    1
TensorLy_Tucker_eeg-1_pytorch_symeig_svd_svd       Tucker                eeg-1      1.184237e+10                        True      svd     symeig_svd    1
TensorLy_Tucker_image-0_pytorch_symeig_svd_random  Tucker                image-0    0.000000e+00                        False     random  symeig_svd    1
TensorLy_Tucker_image-0_pytorch_symeig_svd_svd     Tucker                image-0 

In [84]:
mask = (
        (
                dfs_errors_by_decomposition_method['Tucker']['init'] == 'svd'
        )
)

dfs_errors_by_decomposition_method['Tucker'].loc[
    mask
].value_counts(dropna=False)

method_name                                         decomposition_method  data_type  memory_tried_to_allocated_error_mb  is_error  init  svd           
TensorLy_Tucker_eeg-0_pytorch_randomized_svd_svd    Tucker                eeg-0      0.000000e+00                        False     svd   randomized_svd    1
TensorLy_Tucker_eeg-0_pytorch_symeig_svd_svd        Tucker                eeg-0      3.322946e+09                        True      svd   symeig_svd        1
TensorLy_Tucker_video-2_pytorch_symeig_svd_svd      Tucker                video-2    4.665344e+04                        True      svd   symeig_svd        1
TensorLy_Tucker_video-2_pytorch_randomized_svd_svd  Tucker                video-2    0.000000e+00                        False     svd   randomized_svd    1
TensorLy_Tucker_video-1_pytorch_truncated_svd_svd   Tucker                video-1    0.000000e+00                        False     svd   truncated_svd     1
TensorLy_Tucker_video-1_pytorch_symeig_svd_svd      Tucker     

## данные

In [85]:
dfs_by_decomposition_method.keys()

dict_keys(['Tucker', 'TensorTrain', 'CP'])

In [86]:
dfs_by_decomposition_method_encoded.keys()

dict_keys(['Tucker', 'TensorTrain', 'CP'])

### Tucker

In [87]:
mask = dfs_by_decomposition_method['Tucker']['data_type'].isin(['image-0', 'image-1', 'image-2'])
df_tucker_images = dfs_by_decomposition_method['Tucker'].loc[mask]
df_tucker_images

,method_name,decomposition_method,data_type,backend,gpu_allocated_memory_used_mb_min,gpu_allocated_memory_used_mb_median,gpu_allocated_memory_used_mb_mean,gpu_allocated_memory_used_mb_max,ram_mem_used_mb_min,ram_mem_used_mb_median,...,frobenius_error_min,frobenius_error_median,frobenius_error_mean,frobenius_error_max,compression_ratio_min,compression_ratio_median,compression_ratio_mean,compression_ratio_max,init,svd
0,TensorLy_Tucker_image-0_pytorch_truncated_svd_svd,Tucker,image-0,pytorch,19.552246,19.678223,20.332129,23.048828,6175.277344,6184.375000,...,0.672185,0.672185,0.672185,0.672185,50.166931,50.166931,50.166931,50.166931,svd,truncated_svd
1,TensorLy_Tucker_image-0_pytorch_truncated_svd_...,Tucker,image-0,pytorch,14.450195,14.621094,14.709375,14.927246,6191.238281,6200.539062,...,0.672281,0.672281,0.672281,0.672281,50.166931,50.166931,50.166931,50.166931,random,truncated_svd
2,TensorLy_Tucker_image-0_pytorch_symeig_svd_random,Tucker,image-0,pytorch,14.450195,14.621094,14.709375,14.927246,6222.480469,6222.605469,...,0.672281,0.672281,0.672281,0.672281,50.166931,50.166931,50.166931,50.166931,random,symeig_svd
3,TensorLy_Tucker_image-0_pytorch_randomized_svd...,Tucker,image-0,pytorch,18.642090,18.642090,18.951270,19.415039,6271.574219,6271.574219,...,0.672679,0.672679,0.672679,0.672679,50.166931,50.166931,50.166931,50.166931,svd,randomized_svd
4,TensorLy_Tucker_image-0_pytorch_randomized_svd...,Tucker,image-0,pytorch,14.578613,14.927246,14.796289,14.927246,6273.535156,6273.535156,...,0.672281,0.672281,0.672281,0.672281,50.166931,50.166931,50.166931,50.166931,random,randomized_svd
5,TensorLy_Tucker_image-1_pytorch_truncated_svd_svd,Tucker,image-1,pytorch,16.817383,16.817383,16.817383,16.817383,6273.597656,6273.597656,...,2.446034,2.446034,2.446034,2.446034,50.413535,50.413535,50.413535,50.413535,svd,truncated_svd
6,TensorLy_Tucker_image-1_pytorch_truncated_svd_...,Tucker,image-1,pytorch,14.342773,14.342773,14.349512,14.376465,6273.597656,6273.597656,...,2.452591,2.452591,2.452591,2.452591,50.413535,50.413535,50.413535,50.413535,random,truncated_svd
7,TensorLy_Tucker_image-1_pytorch_symeig_svd_random,Tucker,image-1,pytorch,14.342773,14.342773,14.349512,14.376465,6273.597656,6273.601562,...,2.452591,2.452591,2.452591,2.452591,50.413535,50.413535,50.413535,50.413535,random,symeig_svd
8,TensorLy_Tucker_image-1_pytorch_randomized_svd...,Tucker,image-1,pytorch,14.822266,14.822266,14.860449,15.013184,6273.601562,6273.601562,...,2.449736,2.449736,2.449736,2.449736,50.413535,50.413535,50.413535,50.413535,svd,randomized_svd
9,TensorLy_Tucker_image-1_pytorch_randomized_svd...,Tucker,image-1,pytorch,14.342773,14.342773,14.342773,14.342773,6273.601562,6273.601562,...,2.452591,2.452591,2.452591,2.452591,50.413535,50.413535,50.413535,50.413535,random,randomized_svd


### CP

In [88]:
dfs_by_decomposition_method['CP']['init'].value_counts(dropna=False)

init
random    4710
Name: count, dtype: int64

## Tensor Train

In [89]:
mask = dfs_by_decomposition_method['TensorTrain']['data_type'].isin(['eeg-0', 'eeg-1'])
df_tensortrain_eegs = dfs_by_decomposition_method['TensorTrain'].loc[mask]
df_tensortrain_eegs

,method_name,decomposition_method,data_type,backend,gpu_allocated_memory_used_mb_min,gpu_allocated_memory_used_mb_median,gpu_allocated_memory_used_mb_mean,gpu_allocated_memory_used_mb_max,ram_mem_used_mb_min,ram_mem_used_mb_median,...,duration_max,frobenius_error_min,frobenius_error_median,frobenius_error_mean,frobenius_error_max,compression_ratio_min,compression_ratio_median,compression_ratio_mean,compression_ratio_max,svd
52,TensorLy_TensorTrain_eeg-0_pytorch_randomized_svd,TensorTrain,eeg-0,pytorch,2711.777344,2711.777344,2711.777344,2711.777344,3048.007812,3048.007812,...,18.317027,0.020400,0.023498,0.023272,0.025882,52.237849,52.237849,52.237849,52.237849,randomized_svd
53,TensorLy_TensorTrain_eeg-0_pytorch_truncated_svd,TensorTrain,eeg-0,pytorch,2267.629395,2268.627441,2268.228223,2268.627441,3135.894531,3135.894531,...,2.495926,0.026180,0.026180,0.026180,0.026180,52.237849,52.237849,52.237849,52.237849,truncated_svd
54,TensorLy_TensorTrain_eeg-1_pytorch_randomized_svd,TensorTrain,eeg-1,pytorch,3940.248047,3940.615234,3940.541797,3940.615234,3192.355469,3192.421875,...,32.316716,0.027143,0.027377,0.027353,0.027484,49.941666,49.941666,49.941666,49.941666,randomized_svd
55,TensorLy_TensorTrain_eeg-1_pytorch_truncated_svd,TensorTrain,eeg-1,pytorch,3529.966309,3529.966309,3530.094727,3530.560547,3239.156250,3239.156250,...,49.533440,0.049401,0.049401,0.049401,0.049401,49.941666,49.941666,49.941666,49.941666,truncated_svd
